## DMX Transmitter

In [1]:
devices = %mpy --list
ports = devices.fields(0)

for port in ports:
    s = %mpy --info --select {port}
    print(s)
    
PICO = 'COM15'
PICO_W = 'COM4'

SENDER = PICO
RECEIVER = PICO_W

%mpy --select {SENDER}

{'ver': 'v1.21.0', 'version': '1.21.0', 'port': 'rp2', 'mpy': 'v6.1', 'build': '', 'cpu': 'RP2040', 'family': 'micropython', 'board': 'Raspberry Pi Pico W with RP2040', 'arch': 'armv6m', 'serial_port': 'COM4'}
{'ver': 'v1.21.0', 'version': '1.21.0', 'port': 'rp2', 'mpy': 'v6.1', 'build': '', 'cpu': 'RP2040', 'family': 'micropython', 'board': 'Raspberry Pi Pico with RP2040', 'arch': 'armv6m', 'serial_port': 'COM15'}


In [2]:
from typing_extensions import TYPE_CHECKING  # type: ignore

if TYPE_CHECKING:
    from rp2.asm_pio import *

In [3]:
# %%micropython --select {SENDER} --reset
from rp2 import PIO

# remove all programs from both PIOs
for n in (0, 1):
    pio = PIO(n)
    pio.remove_program()

True


In [4]:
# %%micropython  --select {SENDER}
from machine import Pin

sig_max485_send = Pin(13, Pin.OUT, Pin.PULL_DOWN)

sig_max485_send.on()
x = 1
sig_max485_send.off()

In [5]:
# %%micropython --select {SENDER}
# Example using PIO to create a UART TX interface

from machine import Pin, Signal
from rp2 import PIO, StateMachine, asm_pio

# -----------------------------------------------
# Wiring schema for the DMX TX

pin_dmx_tx = Pin(15, Pin.OUT)  # send data to the DMX bus
pin_dmx_rx = Pin(14, Pin.IN, pull=Pin.PULL_DOWN)  # receive data from the DMX bus
sig_max485_send = Pin(12, Pin.OUT, Pin.PULL_DOWN)  # enable/disable the MAX485 chip

p1 = Pin(1, Pin.OUT, value=0)  # Debugging aid to sync view on the the logic analyzer


@asm_pio(
    set_init=PIO.OUT_LOW,
    sideset_init=PIO.OUT_LOW,
    out_init=PIO.OUT_LOW,
    # autopull=True,  # ?
    out_shiftdir=PIO.SHIFT_RIGHT,  # ?
    push_thresh=8,
)
# fmt: off
def dmx_send():
    """
    - valid values for the data channels are 0-255
    Minimum universe length is 10 channels / slots

    
    The pio Statemachine must be reset before sending a new universe of data,
    this will send the BREAK and the MAB signals
    the universe is an bytearray of a 1 + 512 bytes
    The first byte is the START_CODE , the next 512 bytes are the data channels
    - the START_CODE is 0x00 for DMX Data Packets
    - the START_CODE is 0xCC for RDM Data Packets
    - the START_CODE is 0xFE for RDM Discovery Data Packets


    """
    # Assert break condition
    # Sender DMX break signal is 92us >, so we need to loop at least 92 / 8 = 12 times
    # TODO: Actually should be at least 92us
    set(x, 21)      .side(0)                # BREAK condition for 176us

    label("breakloop")                      # Loop X times, each loop iteration is 8 cycles.
    jmp(x_dec, "breakloop")             [7] # Each loop iteration is 8 cycles.

    nop()                    .side(1)   [7] # Assert MAB. 8 cycles nop and 8 cycles stop-bits = 16us

    # Send data frame
    wrap_target()
    
    pull()                   .side(1)   [7] # 2 STOP bits,  1 + 7 clocks,   or stall with line in idle state (extending MAB) 
    set(x, 7)                .side(0)   [3] # 1 START BIT  1 + 4 clocks load bit counter, assert start bit for 4 clocks

    label("bitloop")
    out(pins, 1)                        [2]  # Shift 1 bit from OSR to the first OUT pin
    jmp(x_dec, "bitloop")                    # Each loop iteration is 4 cycles.

    wrap()
# fmt: on


# Usage example
sm_dmx_tx = StateMachine(
    1,
    dmx_send,
    freq=1_000_000,
    set_base=pin_dmx_tx,
    out_base=pin_dmx_tx,
    sideset_base=pin_dmx_tx,
)
sm_dmx_tx.active(1)

In [6]:
CAPTURE = False

if CAPTURE:
    # start capture

    # setup logic capture
    from saleae import automation


    manager = automation.Manager.connect()

    # setup logic capture on all channels

    device_configuration = automation.LogicDeviceConfiguration(

        enabled_digital_channels=[0, 1, 2, 3, 4, 5, 6, 7],

        digital_sample_rate=12_000_000,

    )

    capture = manager.start_capture(device_configuration=device_configuration)

In [7]:
# %%micropython  --select {SENDER} --no-follow
p1.on()
import time

size = 512
from array import array

universe = array("B", [0] + [255] * (size))  # 1 start code + 512 channels

# print(f"{len(universe)=} {universe=}")

for i in range(len(universe)):
    universe[i] = i % 256

universe[0] = 123  # test Start Code

# for n in range(2):
while 1:
    sig_max485_send.on()
    sm_dmx_tx.restart()
    sm_dmx_tx.active(1)

    sm_dmx_tx.put(universe)

    time.sleep_us(4 * 50)  # wait for the last 4 frames to be sent before switching  the 485 driver
    sig_max485_send.off()
    time.sleep_ms(300)

p1.off()

In [8]:
if CAPTURE:
    # End capture

    capture.stop()

    # "Serial", "Accept DMX-1986 4us MAB"

    # Serial = input channel

    capture.add_analyzer(
        "DMX-512", label="DMX TX", settings={"Serial": 2, "Accept DMX-1986 4us MAB": True}
    )
    capture.add_analyzer(
        "DMX-512", label="DMX RX", settings={"Serial": 6, "Accept DMX-1986 4us MAB": True}
    )


    # capture.add_analyzer("DMX-512-RX", settings={"Serial": 4, "Accept DMX-1986 4us MAB": True})

    # "Input Channel", "Bit Rate (Bits/s)", "Bits per Frame", "Stop Bits", "Parity Bit", "Significant Bit", "Signal inversion", "Mode"

    # capture.add_analyzer("Async Serial", settings={"Input Channel": 6, "Bit Rate (Bits/s)": 115200})